In [22]:
import random
import json
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import pandas as pd
from tabulate import tabulate

%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Player.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Street.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Utility.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Station.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Chance.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Tax.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Go.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Jail.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\FreeParking.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\GoToJail.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\CommunityChest.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\MonopolyBoard.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\Strategy.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\Simulation Classes\\RandomStrategy.py"

%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\MCTS\\MonopolyBoardMCTS.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\MCTS\\Node.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\MCTS\\MCTS.py"
%run -i "C:\\Users\\cathe\\OneDrive\\Documents\\Capstone - Optimising Monopoly Gameplay Strategies\\MCTS\\State.py"

In [23]:
def game(max_iterations, exploration_weight, max_simulations, max_rounds):
    # create board and add players
    board = MonopolyBoardMCTS()
    player1 = Player('Cat')
    player2 = Player('Vlad')
    board.add_agent(player1)
    board.add_other_player(player2)

    # using a random strategy
    #strategy = RandomStrategy()
    #board.strategy = strategy

    # run MCTS game
    root_state = State()
    root_state.from_monopoly_board(board)
    mcts = MCTS(root_state, max_iterations=max_iterations, exploration_weight=exploration_weight, max_simulations=max_simulations)
    mcts.run_game_quiet(max_rounds=max_rounds)

    return mcts.root.state

In [26]:
from tqdm import tqdm
def grid_search(max_iterations_lst, exploration_weight_lst, max_simulations_lst, max_rounds=20, num_tests=100):
    df = pd.DataFrame(columns=['max_iterations', 'exploration_weight', 'max_simulations', 
        'Rounds', 'Agent', 'Other player(s)', 'Properties', 'Stations', 'Utilities', 'Agent Wealth', 'Other Player(s) Wealth'])
        
    for max_iterations in max_iterations_lst:
        for exploration_weight in exploration_weight_lst:
            for max_simulations in max_simulations_lst:
                print(f'Currently testing max_iterations={max_iterations}, exploration_weight={exploration_weight}, max_simulations={max_simulations}')
                for _ in tqdm(range(num_tests)):
                    state = game(max_iterations, exploration_weight, max_simulations, max_rounds)
                    new_row = {'max_iterations': max_iterations, 'exploration_weight': exploration_weight, 
                        'max_simulations': max_simulations, 'Rounds': state.rounds, 'Agent': state.agent, 
                        'Other player(s)': state.other_players, 'Properties': state.properties, 'Stations': state.stations, 
                        'Utilities': state.utilities, 'Agent Wealth': state.agent_wealth, 
                        'Other Player(s) Wealth': state.other_players_wealth}
                    df.loc[len(df)] = new_row

    return df

In [29]:
random.seed(42)
max_iterations_lst = [50, 100, 250, 500, 1000]
exploration_weight_lst = [1, 1.41]
max_simulations_lst = [5, 10, 25, 50]
grid_search_df = grid_search(max_iterations_lst, exploration_weight_lst, max_simulations_lst)

Currently testing max_iterations=50, exploration_weight=1, max_simulations=5


 35%|███▌      | 35/100 [01:18<01:25,  1.32s/it]

In [ ]:
file_path = 'GridSearch_BaseStrategy.json'
with open(file_path, 'w') as json_file:
    json.dump(grid_search_df, json_file, indent=4)

In [ ]:
file_path = 'GridSearch_BaseStrategy.json'
with open(file_path, 'r') as json_file:
    grid_search_df = json.load(json_file)